## Introducción Scraping

Trabajo de scraping sobre la web de FIFA para obtener información de los jugadores de la selección española de fútbol en el Mundial de 2022

https://www.fifa.com/fifaplus/es/tournaments/mens/worldcup/qatar2022/teams/spain/squad


In [ ]:
! pip install lxml bs4 requests

In [1]:
from bs4 import BeautifulSoup

html_doc = open('spain.html').read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [2]:
jugadores = soup.select('div[class*="player-badge-card_badgeCard"]')
print(len(jugadores))

26


In [3]:
j = jugadores[0]
type(j)

bs4.element.Tag

In [13]:
for n in j.find('div', class_="player-badge-card_playerInfo__3Fy6Q").find_all('span', class_='d-none'):
    print(n.text)

res = j.select('div[class*="player-badge-card_playerInfo"]')
for r in res:
    for n in r.find_all('span', class_='d-none'):
        print(n.text)

    

Robert
Lynch Sánchez
Robert
Lynch Sánchez


In [ ]:
# imagen
j.find('div', class_='player-badge-card_playerImage__301X0').attrs['style']

In [ ]:
import re
re.search(r'url\("(.*)"\)', j.find('div', class_='player-badge-card_playerImage__301X0').attrs['style']).group(1)

In [ ]:
j.find('div', class_='player-badge-card_playerPosition__wjnoI').find('span', class_='d-none').text


In [ ]:
# manager-badge-card_badgeCard__1nDB7


# soup.find_all(lambda tag:tag.name=="div" and hasattr(tag, "class") and "quote" in tag['class'])
j.find('div', class_='player-badge-card_playerPosition')
j.find_all(lambda tag:tag.name=='div' and [ x for x in tag['class'] if x.startswith('player-badge-card_playerPosition')])

In [ ]:
j.select('div[class*="player-badge-card_playerPosition"]')